<a href="https://colab.research.google.com/github/mimrancomsats/ProgrammingforAI_SPRING25/blob/main/Lab_11_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis (Text Classification)**
*   **Text Cleaning**
*   **Text Preprocessing**
*   **Feature Engineering**
*   **ML Model**

# **Importing Preprocessing Libraries**

In [ ]:
!pip install contractions

import pandas as pd
import string

import re
import contractions
import nltk
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer


#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt_tab')


#stopwords.words('english')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 6.1 MB/s eta 0:00:00


# **Reading Data**

In [ ]:
temp_df = pd.read_csv('/content/drive/MyDrive/Programming for AI_SPRING-25/Labs/IMDB Dataset.csv')
df = temp_df.iloc[:50000]

# **Text Cleaning & Preprocessing**

In [ ]:
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)

def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_contractions(text):
  expanded_text = contractions.fix(text)
  return expanded_text

def remove_punc(text):
    unwanted = set(string.punctuation + string.digits)
    return ''.join(char for char in str(text) if char not in unwanted)

In [ ]:
df['review'] = df['review'].str.lower()

df['review'] = df['review'].apply(remove_html_tags)

df['review'] = df['review'].apply(remove_url)

#df['review'] = df['review'].apply(remove_contractions)

#df['review'] = df['review'].apply(remove_punc)

#df['review'] = df['review'].apply(word_tokenize)

#df['review'] = df['review'].apply(remove_stopwords)

#df['review'] = df['review'].apply(lemmatize_words)

In [ ]:
df['review']

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is..."
...,...
49995,i thought this movie did a down right good job...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,i am a catholic taught in parochial elementary...
49998,i'm going to have to disagree with the previou...


# **Feature Engineering**

**Target Column Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder

#X = df.drop('sentiment', axis=1)
X = df['review']
Y = df['sentiment']

print(X)
print(Y)

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

print(Y)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object
0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object
[1 1 1 ... 0 0 0]


**Bag of Words**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

print(X_train.shape)
#print(X_train.head)

#print(X_train)
#print(X_test)

vectorizer = CountVectorizer()

X_train_bow = vectorizer.fit_transform(X_train)

X_test_bow = vectorizer.transform(X_test)

# Output the shapes of the resulting Bag of Words matrices
print(f"Shape of X_train_bow: {X_train_bow.shape}")
print(f"Shape of X_test_bow: {X_test_bow.shape}")

# Applying Random Forest Classifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
#accuracy_score(y_test,y_pred)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

(40000,)
Shape of X_train_bow: (40000, 94718)
Shape of X_test_bow: (10000, 94718)
0.8545
[[4245  716]
 [ 739 4300]]


**n-gram (2-gram)**

In [ ]:
cv = CountVectorizer(ngram_range=(2,2))

X_train_n_gram = cv.fit_transform(X_train)
X_test_n_gram = cv.transform(X_test)

# Output the shapes of the resulting Bag of Words matrices
print(f"Shape of X_train_bow: {X_train_n_gram.shape}")
print(f"Shape of X_test_bow: {X_test_n_gram.shape}")

rf = RandomForestClassifier()

rf.fit(X_train_n_gram,y_train)
y_pred = rf.predict(X_test_n_gram)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

Shape of X_train_bow: (40000, 2028692)
Shape of X_test_bow: (10000, 2028692)
0.8535
[[4214  747]
 [ 718 4321]]


**TF/IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Output the shapes of the resulting Bag of Words matrices
print(f"Shape of X_train_bow: {X_train_tfidf.shape}")
print(f"Shape of X_test_bow: {X_test_tfidf.shape}")

rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

Shape of X_train_bow: (40000, 94718)
Shape of X_test_bow: (10000, 94718)
0.8473
[[4254  707]
 [ 820 4219]]


# **Tasks:**
*   **Add a Python Function to remove Stop Words from the IMDB reviews data.**
*   **After Stopword Removal, add a Python Function to perform Lemmitization over IMDB Reviews data.**

**After applying the above mentioned data preprocessing steps, again run this code and analyse the performance of the ML models for text classification of IMDB Reviews.**

**Apply the ML classifier on the following dataset. https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset**


